In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import astropy.coordinates as asc

In [2]:
path = "/media/darkwake/VIB2/Project-IceCube/icecube_10year_ps/events"
filenames = ["IC40_exp.csv", "IC59_exp.csv","IC79_exp.csv", "IC86_I_exp.csv", "IC86_II_exp.csv",
 "IC86_III_exp.csv", "IC86_IV_exp.csv", "IC86_V_exp.csv", "IC86_VI_exp.csv", "IC86_VII_exp.csv"]

In [3]:
file = filenames[0]
f = open(os.path.join(path, file), 'r')

data = [0,0,0,0]
lines = f.readlines()
column=lines[0].split()
column.pop(0)
f.close()

In [4]:
#ICECUBE DATA
content = []

for file in filenames:
    f = open(os.path.join(path, file), 'r')
    lines = f.readlines()
    for line in lines[1:]:
        content.append(line.split())
    f.close()



In [5]:
icdata = pd.DataFrame(content, columns=column)
icdata

,MJD[days],log10(E/GeV),AngErr[deg],RA[deg],Dec[deg],Azimuth[deg],Zenith[deg]
0,54562.37911273,3.31,0.48,203.024,16.760,218.374,106.712
1,54562.38662244,5.17,0.62,75.024,-13.371,349.100,76.646
2,54562.38789162,3.92,0.56,97.501,11.290,327.060,101.291
3,54562.39267689,5.57,1.13,305.857,-33.932,120.413,56.090
4,54562.40477593,5.80,1.11,143.516,-38.279,287.171,51.684
...,...,...,...,...,...,...,...
1134445,58307.96110988,5.14,0.20,221.246,-59.959,141.239,29.956
1134446,58307.96266638,2.94,1.23,191.547,4.415,171.627,94.312
1134447,58307.96355617,2.88,0.30,308.127,42.543,55.437,132.606
1134448,58307.96397167,2.98,0.21,332.199,44.720,31.489,134.811


In [6]:
#MS_PULSAR
f = open("10milsecpsr.txt", 'r')
lines = f.readlines()

content=[]
column=lines[3].split()

for line in lines[:]:
    content.append(line.split())

#the INITAL DATABASE IS CLUTTERED SO WE REMOVE THE NULL COLUMNS AND OTHER CLUTTER
mspdata = pd.DataFrame(content).drop(range(0,6)).dropna().drop([2,6,8,10,11,13,14], axis=1)
f.close()

mspdata.columns = column
mspdata


,#,NAME,Gl,Gb,RAJD,DECJD,P0,F0,DIST
6,1,J0023+0923,111.383,-52.849,5.82032291,9.38996121,0.003050203104754390,327.8470205611185,1.818
7,2,J0024-7204aa,305.895,-44.889,6.02363,-72.08128,0.00184,543.478261,2.688
8,3,J0024-7204ab,305.891,-44.891,6.034006,-72.0798894,0.0037046394947985,269.93179806134,2.540
9,4,J0024-7204ac,305.898,-44.902,6.02363,-72.06806,0.00274,364.963504,4.690
10,5,J0024-7204ad,305.898,-44.902,6.02363,-72.06806,0.00374,267.379679,4.690
...,...,...,...,...,...,...,...,...,...
560,463,J2310-0555,69.702,-57.907,347.526792,-5.92664,0.0026125867919415,382.76240356282,1.556
561,464,J2317+1439,91.361,-42.360,349.288484923,14.65868362,0.0034452510723611,290.25460815391,1.667
562,465,J2322+2057,96.515,-37.310,350.5930632,20.9507437,0.0048084282894641,207.96816335831,1.011
564,466,J2322-2650,28.637,-70.228,350.64433350,-26.84953253,0.00346309917908790,288.758695112907,0.227


### HAVERSINE METHOD TO GET THE SPACE ANGLE BETWEEN TWO POINTS GIVEN (RA,DEC)

def cel_ang(n_ra, n_dec, p_ra, p_dec):

   n_ra = np.deg2rad(n_ra)                     
   n_dec = np.deg2rad(n_dec)     
   p_ra = np.deg2rad(p_ra)                       
   p_dec = np.deg2rad(p_dec)  

   d = np.sin((p_ra - n_ra)/2)**2 + np.cos(n_ra)*np.cos(p_ra) * np.sin((p_dec - n_dec)/2)**2

   return 2 * np.arcsin(np.sqrt(d))

### VECTORIZED HAVERSINE METHOD

In [7]:
def hvvec(lon1, lat1, lon2, lat2):

    #Convert decimal degrees to Radians:
    lon1 = np.radians(lon1)
    lat1 = np.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)

    #Implementing Haversine Formula: 
    dlon = np.subtract(lon2, lon1)
    dlat = np.subtract(lat2, lat1)

    a = np.add(np.power(np.sin(np.divide(dlat, 2)), 2),  
                          np.multiply(np.cos(lat1), 
                                      np.multiply(np.cos(lat2), 
                                                  np.power(np.sin(np.divide(dlon, 2)), 2))))
    c = np.multiply(2, np.arcsin(np.sqrt(a)))

    return c

HAVERSINE WITHOUT APPROXIMATIONS

In [17]:
def hvovec(lon1, lat1, lon2, lat2):

    #Convert decimal degrees to Radians:
    lon1 = np.radians(lon1)
    lat1 = np.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)

    #Implementing Haversine Formula: 
    dlon = np.subtract(lon2, lon1)
    #dlat = np.subtract(lat2, lat1)

    a = np.add(np.multiply(np.sin(lat1), np.sin(lat2)), np.multiply(np.multiply(np.cos(lat1), np.cos(lat2)), np.cos(dlon)))

    return np.arccos(a)

In [18]:
icra = [float(icdata['RA[deg]'][i]) for i in range(0,len(mspdata))]
icdec = [float(icdata['Dec[deg]'][i]) for i in range(0,len(mspdata))]
msra = [float(i) for i in mspdata['RAJD']]
msdec = [float(i) for i in mspdata['DECJD']]
msreq = np.array(msra) - np.array(msdec)
len(msra)

441

ASTROPY METHOD
#ASTROPY METHOD
asp = asc.position_angle( np.deg2rad(msra), np.deg2rad(msdec), np.deg2rad(icra), np.deg2rad(icdec))

In [20]:
hvs2 = hvvec(np.array(msra), np.array(msdec), np.array(icra), np.array(icdec))

In [21]:
hvos = hvovec(np.array(msra), np.array(msdec), np.array(icra), np.array(icdec))

In [22]:
np.rad2deg(hvs2[5])

55.49659028463874

55.49659028463874

np.rad2deg(asp[5].value)

162.1432 (angle between neutrino and pulsar)

295.8124(angle between pulsar and neutrino)





.

In [24]:
np.rad2deg(hvos[5])

55.496590284638735






## IGNORE

print(f'{icra[5]}, {icdec[5]}\n {msra[5]}, {msdec[5]}')

icra = [float(i) for i in icdata['RA[deg]']]
icdec = [float(i) for i in icdata['Dec[deg]']]
float(len(icra))/float(len(msra))

icra_s = np.array_split(icra[:len(icra)-198], 2572)
icdec_s = np.array_split(icdec[:len(icra)-198],2573)

len(icra_s[125])

#icra_s[2571].append(0)